# Objectiv example notebook

This notebook is a lightweight example to show what Objectiv can do for data collection & modeling. There is also a [web version ](https://notebook.objectiv.io/lab?path=product_analytics.ipynb) that contains real data from our website as an example.

Once you have instrumented your website or app with the Objectiv tracker and have a PostgreSQL storage running, you can use this notebook to connect to that. The metrics are just an initial example of what Objectiv can do without the need for additional transformation or cleaning of data beforehand. Have a look at our [docs](https://docs.objectiv.io) for the full rundown.

For any question, please join our [Slack channel](https://join.slack.com/t/objectiv-io/shared_invite/zt-u6xma89w-DLDvOB7pQer5QUs5B_~5pg).

In [ ]:
import sys
import datetime
import plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import sqlalchemy

from jupyter_dash import JupyterDash as Dash
from sankey_dash import get_app

# import Objectiv Bach
from bach import DataFrame
sys.path.extend([
    '../../bach',
    '../'
])

from objectiv_bach.util import duplo_basic_features

from bach.series.series_objectiv import FeatureFrame
from bach.series.series_objectiv import SeriesGlobalContexts, SeriesLocationStack
from bach.types import _registry
_registry.register_dtype_series(SeriesGlobalContexts, [], override_registered_types=True)
_registry.register_dtype_series(SeriesLocationStack, [], override_registered_types=True)

## Connect to full dataset in PostgreSQL

In [ ]:
# connect to full postgresql dataset, add database and credentials here
engine = sqlalchemy.create_engine('postgresql://objectiv:@localhost:5432/objectiv')

In [ ]:
# create a Bach dataframe based on the full dataset
basic_features = duplo_basic_features()
df = DataFrame.from_model(engine=engine, model=basic_features, index=['event_id'])

## Set sample / unsample

In [ ]:
# if desired, sample the data to develop models, in this example case we keep it at 100%
df = df.get_sample(table_name='basic_features_sample', sample_percentage=100, overwrite=True)

# NOTE: if you ran this once, you need to put overwrite as True, otherwise error that it already exists.

In [ ]:
# to unsample the data and run all models below on full dataset, use:
# df = df.get_unsampled()

## Add global contexts & location stack

In [ ]:
# add the global contexts and location stack as custom dtype so we can use them in modeling
df['global_contexts'] = df.global_contexts.astype('objectiv_global_context')
df['location_stack'] = df.location_stack.astype('objectiv_location_stack')

# add the event location from the location_stack as new column to the df, using ls function
df['event_location'] = df.location_stack.ls.nice_name

## Set the user application

In [ ]:
# add a new column to df with the user application from the global contexts, using gc function
df['user_application'] = df.global_contexts.gc.application

# select one or more user application(s) for analysis, in this case objectiv.io website 
df = df[(df['user_application'] == 'objectiv-website')]

## TEMP filter on IP

In [ ]:
# add IP as column using gc function and filter on it
df['ip'] = df.global_contexts.gc.get_from_context_with_type_series(type='HttpContext', key='remote_address')

df = df[(df['ip'] != '144.178.74.106') & 
        (df['ip'] != '77.160.42.36') &
        (df['ip'] != '86.86.89.85')]

## Set the time aggregation 

In [ ]:
# choose for which level of time aggregation the rest of the analysis will run
# supports all Postgres datetime template patterns: https://www.postgresql.org/docs/9.1/functions-formatting.html#FUNCTIONS-FORMATTING-DATETIME-TABLE

agg_level = 'YYYYIW'

# add the time aggregation as new column to the dataframes, so we can group on this later
df['time_aggregation'] = df['moment'].format(agg_level)

## Set the timeframe

In [ ]:
# set the timeframe for analysis
timeframe_selector = (df['moment'] >= datetime.date(2021,10,1)) & (df['moment'] < datetime.date(2021,11,8))

# create a new df with timeframe applied 
timeframe_df = df[timeframe_selector]

# explore the data
timeframe_df.sort_values(by='moment', ascending=False).head()

## Users

In [ ]:
# calculate unique users per timeframe
users = timeframe_df.groupby('time_aggregation').aggregate({'user_id':'nunique'})

# calculate total users, to reuse later
total_users = timeframe_df['user_id'].nunique()

users.sort_values(by='time_aggregation', ascending=False).head()

In [ ]:
# visualize users
fig = px.line(data_frame = users.head(60))
fig.show()

## Sessions

In [ ]:
# calculate unique sessions
sessions = timeframe_df.groupby('time_aggregation').aggregate({'session_id':'nunique'})

sessions.sort_values(by='time_aggregation', ascending=False).head()

In [ ]:
# visualize sessions
fig = px.line(data_frame = sessions.head(60))
fig.show()

## Sessions per user

In [ ]:
# merge users and sessions
users_sessions = sessions.merge(users, how='inner', on='time_aggregation')

# calculate average sessions per user
users_sessions['sessions_per_user_avg'] = users_sessions['session_id_nunique'] / users_sessions['user_id_nunique']

# clean-up columns
users_sessions.drop(columns=['session_id_nunique', 'user_id_nunique'], inplace=True)

users_sessions.sort_values('time_aggregation', ascending=False).head()

In [ ]:
# visualize average sessions per user
fig = px.line(data_frame = users_sessions.head(60))
fig.show()

## New users

In [ ]:
# define first seen per user, based on dataset with not timeframe applied
user_first_seen = df.groupby('user_id').aggregate({'time_aggregation':'min', 'session_id':'min'})

# select all users that have been active in the time
active_users = timeframe_df.groupby('user_id').aggregate({'session_id':'nunique'})

# reset index so we can use user_id to join and clean-up columns
active_users = active_users.reset_index()
active_users.drop(columns=['session_id_nunique'], inplace=True)

# merge with users that have been active in the timeframe
user_first_seen = user_first_seen.merge(active_users, how='inner', on='user_id')

# calculate new users for each timeframe
new_users = user_first_seen.groupby('time_aggregation_min').aggregate({'user_id':'nunique'})

# merge with total users to calculate ratio and limit to timerange
new_total_users = users.merge(new_users, how='inner', left_on='time_aggregation', right_on='time_aggregation_min', suffixes=('_total', '_new'))

# set time_aggregation as single index
new_total_users = new_total_users.set_index('time_aggregation')

# calculate new & returning user share
new_total_users['new_user_share'] = new_total_users['user_id_nunique_new'] / new_total_users['user_id_nunique_total']
new_total_users['returning_user_share'] = (new_total_users['user_id_nunique_total'] - new_total_users['user_id_nunique_new']) / new_total_users['user_id_nunique_total']

new_total_users.sort_values(by='time_aggregation', ascending=False).head()

In [ ]:
# visualize new users
fig = px.line(data_frame = new_total_users[['user_id_nunique_new', 'user_id_nunique_total']].head(60))
fig.show()

In [ ]:
# visualize returning users
fig = px.line(data_frame = new_total_users[['returning_user_share']].head(60))
fig.show()

## Feature creation

In [ ]:
# using Objectiv, you can create features that utilize the context of where they occur on the UI, using the location stack

# first, create a feature frame that will be used on create features
feature_frame = FeatureFrame.from_data_frame(bt=timeframe_df, location_stack_column='location_stack', event_column='event_type')
feature_frame.head()

In [ ]:
# using a Dash app, you can visualize all events with the location stack and create features

# as an example, we'll create features:
# 1) the job annoucement bar that is on both Home & About pages, and call these 'working_features'
# 2) a started conversion funnel. called 'conversion_started'
# 4) a completed conversion, called 'conversion_complete'

app = get_app(Dash, feature_frame)
app.run_server(mode='inline', height = 1000, port=8053)

In [ ]:
# here you can see that a new column is added for each feature we created, containing the location stack
feature_frame.head()

In [ ]:
# if you are happy with the result, write these creatured features to the working df
timeframe_df = feature_frame.write_to_full_frame()
timeframe_df.head()

## Features

In [ ]:
# select the features we just created
created_features = timeframe_df[(timeframe_df.working_features.notnull()) | 
                                (timeframe_df.conversion_started.notnull()) |
                                (timeframe_df.conversion_completed.notnull())]

# get the number of total users and hits per feature
users_per_event = created_features.groupby(['time_aggregation', 'event_type', 'event_location']).aggregate({'user_id':'nunique','session_hit_number':'count'})

users_per_event.sort_values(by=['time_aggregation', 'user_id_nunique'], ascending=False).head()

## New user features

In [ ]:
# look at the first 10 things new users do, for the features we created

# get the first session for users that were new in the timeframe
timeframe_new_users = created_features.merge(user_first_seen, how='inner', left_on=['user_id', 'time_aggregation', 'session_id'], right_on=['user_id', 'time_aggregation_min', 'session_id_min'])

# limit to the first 10 events
timeframe_new_users = timeframe_new_users[(timeframe_new_users['session_hit_number'] <= 10)]

# number of total user and hits per feature
new_user_events = timeframe_new_users.groupby(['time_aggregation', 'event_type', 'event_location']).aggregate({'user_id':'nunique','session_hit_number':'count'})

new_user_events.sort_values(by=['time_aggregation', 'user_id_nunique'], ascending=False).head()

## Conversion

In [ ]:
# select the created conversion feature
conversion_completed = timeframe_df[(timeframe_df.conversion_completed.notnull()) &
                                   (timeframe_df['event_type'] == 'CompletedEvent')]

# calculate conversions, now per user, but can easily be aggregated to session_id instead
conversions = conversion_completed.groupby('time_aggregation').aggregate({'user_id':'nunique'})

# merge with users, but can easily be done with sessions instead
conversion_rate = conversions.merge(users, how='inner', on='time_aggregation', suffixes=('_converting', '_total'))

# calculate conversion rate
conversion_rate['conversion_rate'] = conversion_rate['user_id_nunique_converting'] / conversion_rate['user_id_nunique_total']

conversion_rate.sort_values(by='time_aggregation', ascending=False).head()

In [ ]:
# visualize conversion rate
fig = px.line(data_frame = conversion_rate[['conversion_rate']].head())
fig.show()

## Conversion error rate

In [ ]:
# filter on the created conversion start events, this case a click on 'keep me posted' button
conversion_start = timeframe_df[(timeframe_df.conversion_started.notnull()) &
                                (timeframe_df['event_type'] == 'ClickEvent')]

# calculate conversion starts, now per user, but can easily be aggregated to session_id instead
conversion_starts = conversion_start.groupby('time_aggregation').aggregate({'user_id':'nunique'})

# join conversion start & complete events
conversion_totals = conversion_starts.merge(conversions, how='inner', on='time_aggregation')

# rename columns
conversion_totals.rename(columns={'user_id_nunique_x':'users_start','user_id_nunique_y':'users_completed'}, inplace=True)

# calculate error rate by comparing starting and successfully completing a conversion event
conversion_totals['error_rate'] = (conversion_totals['users_start'] - conversion_totals['users_completed']) / conversion_totals['users_start']

conversion_totals.sort_values(by='time_aggregation', ascending=False).head()

## Conversion funnel

In [ ]:
# for users that have a conversion event, select their conversion sessions and session_hit_number of the conversion moment
converting_users = conversion_completed.groupby(['user_id', 'session_id']).aggregate({'session_hit_number':'max'})

# merge with the df that has all user events in the timeframe
converting_users_events = timeframe_df.merge(converting_users, how='inner', on=['user_id', 'session_id'])

# select all events that converting users had up to their conversion moment in the same session
converting_users_events = converting_users_events[(converting_users_events['session_hit_number'] <= converting_users_events['session_hit_number_max'])]

# filter on only ClickEvent so we focus on user interactions
converting_users_events = converting_users_events[(converting_users_events['event_type'] == 'ClickEvent')]

# select all unique features used by these users
converting_users_features = converting_users_events.groupby(['event_location']).aggregate({'event_id':'nunique'})

# sort these features by name
converting_users_features = converting_users_features.sort_values(by='event_location', ascending=True)

# now we switch to Pandas, as the dataset is small enough and allows nice visualisation
feature_id_pd = converting_users_features.to_pandas().reset_index()

# clean-up columns
feature_id_pd.drop(columns=['event_id_nunique'], inplace=True)

# use the index to give each feature a unique id
feature_id_pd['feature_id'] = feature_id_pd.index

# create a window that returns the previous event for each row
window = converting_users_events.sort_values('session_hit_number').window('session_id')
converting_users_events['prev_event_location'] = converting_users_events.event_location.window_lag(window)

# materizalize the df before we apply an expression on window
converting_users_events = converting_users_events.materialize()

# group each unique event by previous unique event
from_to_events = converting_users_events.groupby(['prev_event_location', 'event_location']).aggregate({'event_id':'nunique'})

# now we switch to Pandas, as the dataset is small enough and allows nice visualisation
from_to_events_pd = from_to_events.to_pandas().reset_index()

# merge with the unique id for each prev_feature
sankey_input_pd = from_to_events_pd.merge(feature_id_pd, how='inner', left_on='prev_event_location', right_on='event_location')
sankey_input_pd = sankey_input_pd.rename(columns={'event_location_x':'event_location', 'feature_id':'prev_feature_id'})
sankey_input_pd = sankey_input_pd.drop(columns={'event_location_y'})

# merge with the unique id for each feature
sankey_input_pd = sankey_input_pd.merge(feature_id_pd, how='left', left_on='event_location', right_on='event_location')

# filter out events where prev_feature and feature are the same and user did not go anywhere new
sankey_input_pd = sankey_input_pd[(sankey_input_pd['prev_feature_id'] != sankey_input_pd['feature_id'])]

# sort by feature, so we can add the correct labels in the sankey
#sankey_input = sankey_input.sort_values(by=['feature_id', 'prev_feature_id'], ascending=True)
sankey_input_pd.head()

In [ ]:
# visualize the sankey
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 50,
      thickness = 5,
      line = dict(color = "black", width = 1),
# NOTE: @hendrik maybe we can come up with a better way to cut-off feature name?
# have now added a hover with the full nice name
      label = feature_id_pd['event_location'].str.slice(0,30).tolist(),
      color = "blue",
      customdata = feature_id_pd['event_location'].tolist(),
      hovertemplate='%{customdata}<br />'+
        'has value %{value}<extra></extra>'  
    ),
    link = dict(
      source = sankey_input_pd['prev_feature_id'].tolist(),
      target = sankey_input_pd['feature_id'].tolist(),
      value = sankey_input_pd['event_id_nunique'].tolist(),
  ))])

fig.update_layout(title_text="Conversion funnel", font_size=10)
fig.show()

## Session duration

In [ ]:
# calculate duration of each session
session_duration = timeframe_df.groupby(['session_id']).aggregate({'moment':['min','max'], 'time_aggregation':'min'})
session_duration['session_duration'] = session_duration['moment_max'] - session_duration['moment_min']

# check which sessions have duration of zero and filter these out, as they are bounces
session_duration = session_duration[(session_duration['session_duration'] > '0')]

# rename columns
session_duration.rename(columns={'time_aggregation_min':'time_aggregation'}, inplace=True)

# calculate average session duration
avg_session_duration = session_duration.groupby(['time_aggregation']).aggregate({'session_duration': 'mean'})

avg_session_duration.sort_values(by='time_aggregation', ascending=False).head()

## Session duration for specific features

In [ ]:
# from the features we created, select oen or more to calculate duration for
start_stop = timeframe_df[timeframe_df.working_features.notnull()]

# get previous (because of the sorting) event for stop event _in the same session, window_lag(n) returns the nth previous value in the partition
window = start_stop.sort_values('moment').window('session_id')
start_stop['prev_event'] = start_stop.feature.window_lag(window)
start_stop['prev_moment'] = start_stop.moment.window_lag(window)

# materizalize the df before we apply an expression on window
start_stop = start_stop.materialize()

# calculate duration
start_stop['duration'] = start_stop.moment - start_stop.prev_moment

# calculate average duration per timeframe
duration_between_events = start_stop.groupby('time_aggregation').aggregate({'duration':'sum'})

duration_between_events.sort_values(by='time_aggregation', ascending=False).head()

## Retention

In [ ]:
# select all sorted time aggregations in the timeframe 
time_aggregations = timeframe_df.groupby(['time_aggregation']).aggregate({'user_id':'nunique'}).sort_values(by='time_aggregation', ascending=True)
time_aggregations.head()

# switch to Pandas as the dataset is small enough reset the index, use that to number each cohort
time_cohorts = time_aggregations.to_pandas().reset_index()
time_cohorts['cohort_id'] = time_cohorts.index
time_cohorts.drop(columns=['user_id_nunique'], inplace=True)

# select all active moments for each user
user_moments = timeframe_df.groupby(['user_id', 'time_aggregation']).aggregate({'moment':'count'})

# merge with first seen df
user_activity = user_moments.merge(user_first_seen, how='inner', on='user_id')

# clean-up and rename columns
user_activity.rename(columns={'time_aggregation_min':'new_user_cohort'}, inplace=True)
user_activity.drop(columns=['moment_count'], inplace=True)

# limit new users to the selected timeframe
timeframe_start = timeframe_df['time_aggregation'].min()
user_activity = user_activity[(user_activity['new_user_cohort'] >= timeframe_start)]

# for each new_user_cohort count how many users get back per timeframe
retention_input = user_activity.groupby(['new_user_cohort', 'time_aggregation']).aggregate({'user_id':'nunique'})

# add the size of each new user cohort
cohorts = retention_input.merge(new_users, how='inner', left_on='new_user_cohort', right_on='time_aggregation_min', suffixes=('_active', '_cohort'))

# calculate classic retention (so not rolling retention, where users are required to be active each timeframe)
cohorts['retention'] = cohorts['user_id_nunique_active'] / cohorts['user_id_nunique_cohort']

# now we switch to Pandas, as the dataset is small enough and allows nice visualisation
cohorts_pd = cohorts.to_pandas().reset_index()

# merge with cohorts to lookup the id for each new user cohort
cohorts_pd = cohorts_pd.merge(time_cohorts, how='inner', left_on='new_user_cohort', right_on='time_aggregation')
cohorts_pd.drop(columns=['time_aggregation_y'], inplace=True)
cohorts_pd.rename(columns={'cohort_id':'new_user_cohort_id', 'time_aggregation_x':'time_aggregation'}, inplace=True)

# merge with cohorts to lookup the id for each active user cohort
cohorts_pd = cohorts_pd.merge(time_cohorts, how='inner', on='time_aggregation')
cohorts_pd.rename(columns={'cohort_id':'active_user_cohort_id', 'time_aggregation_x':'time_aggregation'}, inplace=True)

# number the cohort in which users were active vs their new user cohort
cohorts_pd['active_in_timeframe'] = cohorts_pd.active_user_cohort_id - cohorts_pd.new_user_cohort_id

# create typical retention matrix
cohorts_pd.pivot('new_user_cohort', 'active_in_timeframe', 'retention').replace(np.nan, 0)

In [ ]:
# remove timeframe 0 where the new users are all there, for better visualisation
cohorts_pd.drop(cohorts_pd[cohorts_pd.active_in_timeframe == 0].index, inplace=True)

# create retention matrix
retention_pd = cohorts_pd.pivot('new_user_cohort', 'active_in_timeframe', 'retention').replace(np.nan, 0)

# visualize heatmap
plt.figure(figsize=(15,10))
fmt = lambda x,pos: '{:.0%}'.format(x)
retention_heatmap = sns.heatmap(retention_pd, center=1, linewidths=1, square=True, annot=True, fmt=".0%", cbar_kws={'format': FuncFormatter(fmt)})

## Bounce rate

In [ ]:
# gather sessions, hits per timeframe
hits_sessions = timeframe_df[['time_aggregation', 'session_id', 'session_hit_number']]

# calculate hits per session
hits_per_session = hits_sessions.groupby(['time_aggregation', 'session_id']).aggregate({'session_hit_number':'nunique'})

# select sessions with only one hit
hit_selector = (hits_per_session['session_hit_number_nunique'] == 1)
single_hit_sessions = hits_per_session[hit_selector]

# count these single hit sessions per timeframe
bounced_sessions = single_hit_sessions.groupby('time_aggregation').aggregate({'session_id':'nunique'})

# merge with total sessions
bounce_rate = bounced_sessions.merge(sessions, how='inner', on='time_aggregation', suffixes=('_bounce', '_total'))

# calculate bounce rate
bounce_rate['bounce_rate'] = bounce_rate['session_id_nunique_bounce'] / bounce_rate['session_id_nunique_total']

# clean-up columns
bounce_rate.drop(columns=['session_id_nunique_bounce', 'session_id_nunique_total'], inplace=True)

bounce_rate.sort_values(by='time_aggregation', ascending=False).head()

In [ ]:
# visualize bounce rate
fig = px.line(data_frame = bounce_rate[['bounce_rate']].head(60))
fig.show()

## User agent

In [ ]:
# add a new column to df with the user_agent from the global contexts, using gc function
timeframe_df['user_agent'] = timeframe_df.global_contexts.gc.user_agent

# gather overall basic stats grouped per user_agent
user_agent_counts = timeframe_df.groupby(['time_aggregation', 'user_agent']).aggregate({'user_id':'nunique', 'session_id':'nunique'})

# add total users and calculate share per user_agent
user_agent_counts['total_users'] = total_users

# calculate share per user_agent
user_agent_counts['share_of_users'] = user_agent_counts['user_id_nunique'] / user_agent_counts['total_users']

# clean-up colums
user_agent_counts.drop(columns=['total_users'], inplace=True)

user_agent_counts.sort_values(by=['time_aggregation', 'user_id_nunique'], ascending=False).head()

## Referer

In [ ]:
# add a new column to dataframe with the referer from the global contexts, using gc function
timeframe_df['referer'] = timeframe_df.global_contexts.gc.get_from_context_with_type_series(type='HttpContext', key='referer')

# gather overall basic stats grouped per referer
referer_counts = timeframe_df.groupby(['time_aggregation', 'referer']).aggregate({'user_id':'nunique', 'session_id':'nunique'})

# add total users and calculate share per referer
referer_counts['total_users'] = total_users

# calculate share per referer
referer_counts['share_of_users'] = referer_counts['user_id_nunique'] / referer_counts['total_users']

# clean-up colums
referer_counts.drop(columns=['total_users'], inplace=True)

referer_counts.sort_values(by=['time_aggregation', 'user_id_nunique'], ascending=False).head()

## User timeline

In [ ]:
# select the spefic user we want to replay
user_selector = (timeframe_df['user_id'].astype('string') == 'fe2657f1-a08c-4e33-b762-441c2f52855c')

# create df with only this user's events
selected_user_df = timeframe_df[user_selector]

# timeline of this user's events
user_timeline = selected_user_df[['moment','event_type', 'event_location', 'user_agent', 'referer']]

user_timeline.sort_values(by='moment', ascending=True).head()

## Frequency

In [ ]:
# number of total sessions per user
total_sessions_user = timeframe_df.groupby(['user_id']).aggregate({'session_id':'nunique'})

# calculate frequency
frequency = total_sessions_user.groupby(['session_id_nunique']).aggregate({'user_id':'nunique'})

# add total users and calculate share per number of sessions
frequency['share_of_users'] = frequency['user_id_nunique'] / total_users

frequency.sort_values(by='session_id_nunique', ascending=True).head()

In [ ]:
# visualize frequency
fig = px.bar(data_frame = frequency[['share_of_users']].head())
fig.show()

## Recency

In [ ]:
# count the number of active days per user
user_active_check = timeframe_df.groupby(['user_id']).aggregate({'day':'nunique'})

# select all users that had more than one active day
user_active_check = user_active_check[(user_active_check['day_nunique'] > 1)]

# select all active days for each user
user_days = timeframe_df.groupby(['user_id', 'day']).aggregate({'time_aggregation':'min'})

# merge with users that have more than one active day
user_days = user_days.merge(user_active_check, how='inner', on='user_id')

# reset the index so we can use the user_id & day columns
user_days = user_days.reset_index()

# get previous (because of the sorting) day for each user
window = user_days.sort_values('day').window(['user_id'])
user_days['prev_day'] = user_days.day.window_lag(window)

# materizalize the df before we apply an expression on window
user_days = user_days.materialize()

# calculate the number of days between an active day and prev_day
user_days['recency'] = user_days['day'] - user_days['prev_day']

# rename columns
user_days.rename(columns={'time_aggregation_min':'time_aggregation'}, inplace=True)

# calculate the recency per time_aggregation
recency = user_days.groupby(['time_aggregation']).aggregate({'recency':'mean','user_id':'nunique'})

recency.sort_values(by='time_aggregation', ascending=False).head()

## Get metrics to production

In [ ]:
# we're working on export functionality to dbt, until then, you can use view_sql() to get the SQL that runs on the full dataset for any metric above

# as an example, the SQL for the session duration metric
print(avg_session_duration.view_sql())